# Predicción de postulación para un aviso

In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder

## Carga y limpieza de datos / Feature Engineering

In [2]:
# cargo postulantes
df_postulantes1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])
df_postulantes2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])
df_postulantes3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])

df_postulantes = df_postulantes1.append(df_postulantes2).append(df_postulantes3)

del df_postulantes1
del df_postulantes2
del df_postulantes3

df_postulantes.drop_duplicates(['idpostulante'], keep='first', inplace=True)

df_postulantes.shape

(504407, 3)

In [3]:
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# guardamos los codificadores (label => numero y visceversa) en un diccionario
label_encoders = {}

In [4]:
# limpieza de datos de fecha de nacimiento
df_postulantes['fechanacimiento'] = pd.to_datetime(df_postulantes['fechanacimiento'], errors='coerce')

df_postulantes['edad'] = datetime.datetime.now().year - df_postulantes['fechanacimiento'].dt.year
df_postulantes['edad'] = df_postulantes['edad'].fillna(0)

df_postulantes = df_postulantes.drop(['fechanacimiento'], axis=1)

df_postulantes = df_postulantes.loc[(df_postulantes['sexo'] == 'FEM') | (df_postulantes['sexo'] == 'MASC') | (df_postulantes['sexo'] == 'NO_DECLARA')]

# convierto variables categóricas a numéricas
label_encoders['sexo'] = LabelEncoder().fit(['FEM', 'MASC', 'NO_DECLARA'])
df_postulantes['sexo'] = label_encoders['sexo'].transform(df_postulantes['sexo'])

df_postulantes.head()

,idpostulante,sexo,edad
0,NM5M,0,48.0
1,5awk,0,56.0
2,ZaO5,0,40.0
3,NdJl,1,49.0
4,eo2p,1,37.0


In [5]:
# cargo educacion de los estudiantes
df_edu1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_1_postulantes_educacion.csv')
df_edu2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
df_edu3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv')

df_edu = df_edu1.append(df_edu2).append(df_edu3)

del df_edu1
del df_edu2
del df_edu3

df_edu.drop_duplicates(['idpostulante'], keep='first', inplace=True)

print(df_edu.shape)
print(df_edu.head())

(447909, 3)
  idpostulante         nombre    estado
0         NdJl       Posgrado  En Curso
1         8BkL  Universitario  En Curso
2         1d2B  Universitario  En Curso
3         NPBx  Universitario  En Curso
5         Ym2X           Otro  En Curso


In [6]:
# renombro columnas para no confundirlas luego de mergear
df_edu = df_edu.rename(columns={'nombre':'nombre_edu', 'estado': 'estado_edu'});

In [7]:
# convierto variables categóricas a numéricas
label_encoders['nombre_edu'] = LabelEncoder().fit(df_edu['nombre_edu'])
label_encoders['estado_edu'] = LabelEncoder().fit(df_edu['estado_edu'])

df_edu['nombre_edu'] = label_encoders['nombre_edu'].transform(df_edu['nombre_edu'])
df_edu['estado_edu'] = label_encoders['estado_edu'].transform(df_edu['estado_edu'])

df_edu.head()

,idpostulante,nombre_edu,estado_edu
0,NdJl,3,1
1,8BkL,6,1
2,1d2B,6,1
3,NPBx,6,1
5,Ym2X,2,1


In [8]:
df_posts_edu = df_postulantes.merge(df_edu, on='idpostulante')

# me quedo con el de mayor educacion registro para cada postulante
df_posts_edu.sort_values(by='nombre_edu', ascending=False)

df_posts_edu.drop_duplicates(subset = "idpostulante",keep= "first", inplace=True)

df_posts_edu.head()

,idpostulante,sexo,edad,nombre_edu,estado_edu
0,NM5M,0,48.0,4,2
1,5awk,0,56.0,6,2
2,ZaO5,0,40.0,5,2
3,NdJl,1,49.0,3,1
4,eo2p,1,37.0,4,2


In [9]:
# cargo postulaciones
df_postulaciones1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_4_postulaciones.csv')
df_postulaciones2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')

df_postulaciones = df_postulaciones1.append(df_postulaciones2)

del df_postulaciones1
del df_postulaciones2

df_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], keep='first', inplace=True)

print(df_postulaciones.shape)
print(df_postulaciones.head())

(6603752, 3)
      idaviso idpostulante     fechapostulacion
0  1112257047         NM5M  2018-01-15 16:22:34
1  1111920714         NM5M  2018-02-06 09:04:50
2  1112346945         NM5M  2018-02-22 09:04:47
3  1112345547         NM5M  2018-02-22 09:04:59
4  1112237522         5awk  2018-01-25 18:55:03


In [10]:
# no necesitamos la fecha de postulacion
df_postulaciones.drop(['fechapostulacion'], axis=1, inplace=True)
df_postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [11]:
# cargo avisos
df_avisos1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_6_avisos_detalle.csv')
df_avisos2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos4 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

df_avisos = df_avisos1.append(df_avisos2).append(df_avisos3).append(df_avisos4)

del df_avisos1
del df_avisos2
del df_avisos3
del df_avisos4

df_avisos = df_avisos.drop_duplicates(['idaviso'], keep='first')

print(df_avisos.shape)
print(df_avisos.head())

(25288, 11)
      idaviso  idpais                                         titulo  \
0     8725750       1               VENDEDOR/A PROVINCIA DE SANTA FE   
1    17903700       1                                     Enfermeras   
2  1000150677       1                                 Chofer de taxi   
3  1000610287       1  CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA   
4  1000872556       1  Operarios de Planta - Rubro Electrodomésticos   

                                         descripcion        nombre_zona  \
0  <p><strong><strong>Empresa:</strong></strong> ...  Gran Buenos Aires   
1  <p>Solicitamos para importante cadena de farma...  Gran Buenos Aires   
2  <p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...    Capital Federal   
3  <p><strong>Somos una empresa multinacional que...  Gran Buenos Aires   
4  <p><strong>OPERARIOS DE PLANTA</strong></p><p>...  Gran Buenos Aires   

  ciudad       mapacalle tipo_de_trabajo         nivel_laboral nombre_area  \
0    NaN             NaN  

In [12]:
df_avisos = df_avisos.drop(['mapacalle'], axis=1)

In [13]:
# limpieza de NaN, nan, None, etc.
df_avisos['ciudad'] = df_avisos['ciudad'].fillna('None')
df_avisos['titulo'] = df_avisos['titulo'].fillna('None')
df_avisos['descripcion'] = df_avisos['descripcion'].fillna('None')
df_avisos['denominacion_empresa'] = df_avisos['denominacion_empresa'].fillna('None')
df_avisos['nivel_laboral'] = df_avisos['nivel_laboral'].fillna('None')

In [14]:
# convierto variables categóricas a numéricas
label_encoders['nombre_zona'] = LabelEncoder().fit(df_avisos['nombre_zona'])
label_encoders['ciudad'] = LabelEncoder().fit(df_avisos['ciudad'])
label_encoders['tipo_de_trabajo'] = LabelEncoder().fit(df_avisos['tipo_de_trabajo'])
label_encoders['nivel_laboral'] = LabelEncoder().fit(df_avisos['nivel_laboral'])
label_encoders['nombre_area'] = LabelEncoder().fit(df_avisos['nombre_area'])
label_encoders['denominacion_empresa'] = LabelEncoder().fit(df_avisos['denominacion_empresa'])

df_avisos['nombre_zona'] = label_encoders['nombre_zona'].transform(df_avisos['nombre_zona'])
df_avisos['ciudad'] = label_encoders['ciudad'].transform(df_avisos['ciudad'])
df_avisos['tipo_de_trabajo'] = label_encoders['tipo_de_trabajo'].transform(df_avisos['tipo_de_trabajo'])
df_avisos['nivel_laboral'] = label_encoders['nivel_laboral'].transform(df_avisos['nivel_laboral'])
df_avisos['nombre_area'] = label_encoders['nombre_area'].transform(df_avisos['nombre_area'])
df_avisos['denominacion_empresa'] = label_encoders['denominacion_empresa'].transform(df_avisos['denominacion_empresa'])

In [15]:
df_avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,7,30,1,5,30,4005
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,7,30,1,5,158,1640
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,1,30,1,5,181,1561
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,7,30,1,5,181,4119
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,7,30,1,5,143,1267


In [16]:
from math import fmod

def n_grams(string, n):
    if len(string) < n:
        return []
    if len(string) == n:
        return [string]
    ret = []
    for i in range(0, len(string) - n + 1):
        ret.append(string[i:i + n])
    return ret

# sacado de carpeta
# (v & 0xff) = 1er byte de v
# (v >> 8) = desplazo 1 byte hacia la derecha
def jenkins_hash(v):
    h = 0
    while (v & 0xff) != 0:
        byte = (v & 0xff)
        h += byte
        h += (h << 10)
        h ^= (h >> 6)
        v = (v >> 8)
    h += (h << 3)
    h ^= (h >> 11)
    h += (h << 15)
    return h

# apunte pg 230 "Hashing Universal para claves de longitud variable" 
def custom_str_hash(string):
    a = 13
    p = 9941
    h = 0
    for i, c in enumerate(string):
        h = ord(c) * a**i % p 
    return jenkins_hash(h)

# min hashes a la Jaccard
def min_hash(string, n_g):
    h = None
    for gram in n_grams(string, n_g):
        h_gram = custom_str_hash(gram)
        if h == None or h_gram < h:
            h = h_gram
    return h if h != None else 0

In [17]:
vmin_hash = np.vectorize(min_hash)

# numero primo para achicar los min hashes calculados
prime = 235723

# min hashes para los textos (experimental, buscar mejores formas)
df_avisos['minhash_titulo'] = vmin_hash(df_avisos['titulo'], 5) % prime
df_avisos['minhash_descripcion'] = vmin_hash(df_avisos['descripcion'], 5) % prime

df_avisos.drop(['titulo', 'descripcion'], axis=1, inplace=True)

#df_avisos['minhash_titulo'] = np.random.rand(df_avisos.shape[0]) * 100000 % prime
#df_avisos['minhash_descripcion'] = np.random.rand(df_avisos.shape[0]) * 100000 % prime

In [18]:
# merge de todos los datos
df_general = df_postulantes.merge(df_postulaciones, on='idpostulante').merge(df_avisos, on='idaviso')

#del df_postulantes
#del df_postulaciones
#del df_avisos

print(df_general.shape)
print(df_general.head())

(6187927, 13)
  idpostulante  sexo  edad     idaviso  idpais  nombre_zona  ciudad  \
0         NM5M     0  48.0  1112257047       1            7      30   
1        8krKL     1  46.0  1112257047       1            7      30   
2       EDEpY4     0  38.0  1112257047       1            7      30   
3       1lejrO     0  50.0  1112257047       1            7      30   
4       Y0MNYJ     0  41.0  1112257047       1            7      30   

   tipo_de_trabajo  nivel_laboral  nombre_area  denominacion_empresa  \
0                2              4           14                  1369   
1                2              4           14                  1369   
2                2              4           14                  1369   
3                2              4           14                  1369   
4                2              4           14                  1369   

   minhash_titulo  minhash_descripcion  
0          211761                84263  
1          211761                84263  
2  

## Preparación de datos para entrenamiento y predicción

#### Generación de postulaciones

In [19]:
df_general['sepostulo'] = 1

#### Generación de "no" postulaciones

In [21]:
sample = 3000000

In [22]:
df_postulantes_sample = df_postulantes.sample(sample, replace=True).reset_index().drop("index",1)
df_avisos_sample = df_avisos.sample(sample, replace=True).reset_index().drop("index",1)

print(df_postulantes_sample.shape)
print(df_avisos_sample.shape)

(3000000, 3)
(3000000, 10)


In [23]:
df_no_postulaciones = df_postulantes_sample.join(df_avisos_sample)
df_no_postulaciones = df_no_postulaciones.merge(df_postulaciones, on=["idaviso","idpostulante"], how="left")
df_no_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], keep='first', inplace=True)
print(df_no_postulaciones.shape)

(2999678, 13)


In [24]:
# vamos a asumir que no hay postulaciones duplicadas dado que pertenecen a distintos periodos

In [25]:
df_no_postulaciones['sepostulo'] = 0;
df_no_postulaciones.head(5)

,idpostulante,sexo,edad,idaviso,idpais,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,minhash_titulo,minhash_descripcion,sepostulo
0,NzrQD6l,0,62.0,1112129657,1,7,30,1,5,185,1826,53288,181256,0
1,EQ5pO4,0,26.0,1112493483,1,7,30,1,5,185,215,53288,160391,0
2,Bm5OvGv,0,28.0,1112237787,1,7,30,1,4,2,223,181256,181256,0
3,6pR3dR,1,31.0,1112197173,1,7,30,1,5,78,3755,181256,181256,0
4,10x0wE,1,44.0,1112414680,1,7,30,1,1,34,2432,53288,181256,0


In [26]:
print(df_general.shape)
df_general = df_general.append(df_no_postulaciones)
print(df_general.shape)

(6187927, 14)
(9187605, 14)


#### Extracción de datos

In [27]:
# me quedo solo con los datos necesarios para ML
#datos_entrenamiento = df_general[['sexo', 'edad', 'idpais', 'nombre_zona', 'ciudad',
#                               'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
#                               'denominacion_empresa', 'minhash_titulo', 'minhash_descripcion']].values

# target = si se postula o no. Para estos casos es si (True)
#targets_entrenamiento = df_general['sepostulo']

#datos_entrenamiento, targets_entrenamiento = shuffle(datos_entrenamiento, targets_entrenamiento, random_state=13)

#### Carga de datos a predecir

In [28]:
df_test_final = pd.read_csv('../datos_navent_fiuba/test_final_100k.csv')
print(df_test_final.shape)

df_test_final = df_test_final.merge(df_postulantes, on='idpostulante')
print(df_test_final.shape)
df_test_final = df_test_final.merge(df_avisos, on='idaviso')
print(df_test_final.shape)

#datos_prediccion = df_test_final[['sexo', 'edad', 'idpais', 'nombre_zona', 'ciudad',
#                               'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
#                               'denominacion_empresa', 'minhash_titulo', 'minhash_descripcion']].values

(100000, 3)
(100000, 5)
(100000, 14)


## Ejecución del algoritmo de ML

In [29]:
columnas_datos = ['sexo', 'edad', 'idpais', 'nombre_zona', 'ciudad', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa', 'minhash_titulo', 'minhash_descripcion']
columnas_target = ['sepostulo']

In [30]:
df_resultado = pd.DataFrame()
df_resultado['id'] = df_test_final['id']

In [31]:
def guardar_res(df_res, predicciones, algoritmo):
    now = datetime.datetime.now()
    filename = "./submissions/{0}-{1}-{2}.csv".format(algoritmo, now.date(), now.time())
    
    df_res['sepostulo'] = predicciones
    df_res.to_csv(filename, index=False)

### Decision Tree

In [32]:
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
params = { 'max_depth': 1, 'random_state': 29 }

treeclassifier = DecisionTreeClassifier(**params)
treeclassifier.fit(df_general[columnas_datos], df_general[columnas_target])

sepostulo_predicciones = treeclassifier.predict(df_test_final[columnas_datos])

guardar_res(df_resultado, sepostulo_predicciones, "DecisionTree")

### Random Forest

In [1]:
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
params = { 'max_depth': None, 'random_state': 29 }

rndforestclassifier = RandomForestClassifier(**params)
rndforestclassifier.fit(df_general[columnas_datos], df_general[columnas_target].values)

sepostulo_predicciones = rndforestclassifier.predict(df_test_final[columnas_datos])

guardar_res(df_resultado, sepostulo_predicciones, "RandomForest")

NameError: name 'RandomForestClassifier' is not defined

### XGBoost

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
params = {'n_estimators': 3, 'max_depth': 5, 'min_samples_split': 2, 'learning_rate': 0.1 }

xgbclassifier = GradientBoostingClassifier(**params)
xgbclassifier.fit(df_general[columnas_datos], df_general[columnas_target])

sepostulo_predicciones = xgbclassifier.predict(df_test_final[columnas_datos].values)

guardar_res(df_resultado, sepostulo_predicciones, "XGBoost")

### AdaBoost

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
params = {'n_estimators': 1, 'learning_rate': 0.01, 'random_state': 29}

adaboosclassifier = AdaBoostClassifier(**params)
adaboosclassifier.fit(df_general[columnas_datos], df_general[columnas_target])

sepostulo_predicciones = adaboosclassifier.predict(df_test_final[columnas_datos])

guardar_res(df_resultado, sepostulo_predicciones, "AdaBoost")

### SVM (RBF)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
params = {'C': 1.0, 'gamma': 'auto'}

svcclassifier = SVC(**params)
svcclassifier.fit(df_general[columnas_datos], df_general[columnas_target])

sepostulo_predicciones = svcclassifier.predict(df_test_final[columnas_datos])

guardar_res(df_resultado, sepostulo_predicciones, "SVM(RBF)")

### Red Neuronal (Multi-Layer Perceptron)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
params = {'solver': 'lbfgs', 'alpha': 1e-5, 'hidden_layer_sizes': (5, 2), 'random_state': 1}

mlpclassifier = MLPClassifier(**params)
mlpclassifier.fit(df_general[columnas_datos], df_general[columnas_target])

sepostulo_predicciones = mlpclassifier.predict(df_test_final[columnas_datos])

guardar_res(df_resultado, sepostulo_predicciones, "MLP")